# Assignment 9
For this assignment, we will examine our keras MNIST network, and try to optimize it.

Tasks:
1.   Add k-fold validation to your network testing.  It would be best if you put this in a callable module - a skeleton of such code will be given below.
2.   Optimize the network for the number of hidden nodes in the hidden layer: test the following choices: 100,200,400,600,800,1000.   Use validation loss to pick the best choise.
3.   Test the performance of your fully-trained network (after k-fold validation) on the separate test sample.  You will need to retrain on the full sample using the best hiden node choice.   Note that you can put the **test** sample in as the **validation** sample in re-fitting the full network. 
    *  Get the overall performance
    *  Plot the loss and accuracy of the train and test sample versus epoch.
    *  Calculate the confusion matrix on the test sample.

Extra:
How confident is the network when it gets the prediction correct?  How about when it gets the prediction incorrect?   Make a plot which compares these two cases. 

## Get the data and prepare it

In [ ]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

short = False
if short:
    train_images = train_images[:7000,:]
    train_labels = train_labels[:7000]
    test_images = test_images[:3000,:]
    test_labels = test_labels[:3000]
#
print("Train info",train_images.shape, train_labels.shape)
print("Test info",test_images.shape, test_labels.shape)

train_images = train_images.reshape((train_images.shape[0],28*28))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((test_images.shape[0],28*28))
test_images = test_images.astype('float32')/255

from keras.utils import to_categorical

train_labels_cat = to_categorical(train_labels)
test_labels_cat = to_categorical(test_labels)


## Task 1: add k-fold validation using a method

In [ ]:
from keras import models
from keras import layers
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
import numpy as np

def kfold_network(hidden_nodes,activation='tanh',optimizer='adam'):

    network = models.Sequential()
    network.add(layers.Dense(hidden_nodes,activation=activation,input_shape=(28*28,)))
    network.add(layers.Dense(10,activation='softmax'))

    network.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
#
# If we reload this right before fitting the model, the model will start from scratch
    network.save_weights('model_init.h5')
#
# Early stopping
    patienceCount = 10
    callbacks = [EarlyStopping(monitor='val_loss', patience=patienceCount),
                 ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
#
# Add loop over k-folds here

#
# Return the average accuracy and loss (on the validation sample!)
    return avg_acc,avg_loss

## Task 2: Loop over hidden nodes and store results

In [ ]:
#
# Determine best number of hidden nodes
for nodes in [100,200,400,600,800,1000]:
    avg_acc,avg_loss = kfold_network(nodes)


## Task 3: Pick the best model and retrain the network with the full data
Things to keep in mind here:
*  Stop training at the average epoch for the best node choice (don't use early stopping)
*  Use the full training sample
*  Don't include a validation set
*  Remember to save your network